<a href="https://colab.research.google.com/github/musashiCZX/RNN-practice/blob/main/RNN_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import keras
import tensorflow as tf

In [ ]:
def generate_time_series(batch_size, n_steps):
  freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)
  time = np.linspace(0, 1, n_steps)
  series = 0.5 * np.sin((time - offsets1) * (freq1 * 10 + 10)) # wave 1
  series += 0.2 * np.sin((time - offsets2) * (freq2 * 20 + 20)) # + wave 2
  series += 0.1 * (np.random.rand(batch_size, n_steps) - 0.5) # + noise
  return series[..., np.newaxis].astype(np.float32)

In [ ]:
n_steps = 50
series = generate_time_series(10000, n_steps + 1)
X_train, y_train = series[:7000, :n_steps], series[:7000, -1]
X_valid, y_valid = series[7000:9000, :n_steps], series[7000:9000, -1]
X_test, y_test = series[9000:, :n_steps], series[9000:, -1]

In [ ]:
batch_size = 10000
n_step = 50

In [ ]:
freq1, freq2, offsets1, offsets2 = np.random.rand(4, batch_size, 1)

The simplest model with only one neuron

In [ ]:
model = keras.models.Sequential([
keras.layers.SimpleRNN(1, input_shape=[None, 1])
])

More complicated model

In [ ]:
model = keras.models.Sequential([
keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
keras.layers.SimpleRNN(20),
keras.layers.Dense(1)
])

In [ ]:
model.compile(loss = 'mse', metrics=['mae'])

In [ ]:
model.fit(x = X_train, y = y_train, epochs = 20, validation_data = (X_valid, y_valid))

Epoch 1/20
219/219 [==============================] - 33s 140ms/step - loss: 0.0218 - mae: 0.0963 - val_loss: 0.0061 - val_mae: 0.0629
Epoch 2/20
219/219 [==============================] - 29s 135ms/step - loss: 0.0045 - mae: 0.0535 - val_loss: 0.0033 - val_mae: 0.0465
Epoch 3/20
219/219 [==============================] - 31s 140ms/step - loss: 0.0036 - mae: 0.0485 - val_loss: 0.0035 - val_mae: 0.0482
Epoch 4/20
219/219 [==============================] - 32s 144ms/step - loss: 0.0034 - mae: 0.0471 - val_loss: 0.0035 - val_mae: 0.0475
Epoch 5/20
219/219 [==============================] - 30s 138ms/step - loss: 0.0033 - mae: 0.0467 - val_loss: 0.0032 - val_mae: 0.0456
Epoch 6/20
219/219 [==============================] - 30s 139ms/step - loss: 0.0032 - mae: 0.0459 - val_loss: 0.0033 - val_mae: 0.0466
Epoch 7/20
219/219 [==============================] - 31s 142ms/step - loss: 0.0032 - mae: 0.0454 - val_loss: 0.0030 - val_mae: 0.0443
Epoch 8/20
219/219 [==============================] - 3

In [ ]:
series = generate_time_series(1, n_steps + 10)
X_new, Y_new = series[:, :n_steps], series[:, n_steps:]
X = X_new
for step_ahead in range(10):
  y_pred_one = model.predict(X[:, step_ahead:])[:, np.newaxis, :]
  X = np.concatenate([X, y_pred_one], axis=1)
  Y_pred = X[:, n_steps:]

In [ ]:
Y = np.empty((10000, n_steps, 10)) # each target is a sequence of 10D vectors
for step_ahead in range(1, 10 + 1):
  Y[:, :, step_ahead - 1] = series[:, step_ahead:step_ahead + n_steps, 0]
  Y_train = Y[:7000]
  Y_valid = Y[7000:9000]
  Y_test = Y[9000:]

In [ ]:
model = keras.models.Sequential([
keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
keras.layers.SimpleRNN(20, return_sequences=True),
keras.layers.TimeDistributed(keras.layers.Dense(10))
])

In [ ]:
def last_time_step_mse(Y_true, Y_pred):
  return tf.keras.metrics.mean_squared_error(Y_true[:, -1], Y_pred[:, -1])
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
model.compile(optimizer = optimizer, loss = 'mse', metrics = [last_time_step_mse])

In [ ]:
model.fit(x = X_train, y = Y_train, validation_data = (X_valid, Y_valid), epochs = 10)

Epoch 1/10
219/219 [==============================] - 34s 146ms/step - loss: 0.0287 - last_time_step_mse: 0.0430 - val_loss: 0.0025 - val_last_time_step_mse: 0.0024
Epoch 2/10
219/219 [==============================] - 32s 145ms/step - loss: 0.0024 - last_time_step_mse: 0.0048 - val_loss: 8.2350e-04 - val_last_time_step_mse: 8.8166e-04
Epoch 3/10
219/219 [==============================] - 31s 141ms/step - loss: 7.9212e-04 - last_time_step_mse: 0.0014 - val_loss: 5.8537e-04 - val_last_time_step_mse: 9.0580e-04
Epoch 4/10
219/219 [==============================] - 31s 140ms/step - loss: 8.5151e-04 - last_time_step_mse: 0.0019 - val_loss: 5.9144e-04 - val_last_time_step_mse: 7.7548e-04
Epoch 5/10
219/219 [==============================] - 30s 138ms/step - loss: 0.0010 - last_time_step_mse: 0.0027 - val_loss: 5.6143e-04 - val_last_time_step_mse: 9.6929e-04
Epoch 6/10
219/219 [==============================] - 31s 140ms/step - loss: 9.3958e-04 - last_time_step_mse: 0.0025 - val_loss: 4.5359